<a href="https://colab.research.google.com/github/ruka38/daily/blob/master/LLM_8_3_%E5%9F%8B%E3%82%81%E8%BE%BC%E3%81%BF%E3%83%A2%E3%83%87%E3%83%AB%E3%81%AE%E5%AE%9F%E8%A3%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

文埋め込みモデルSimCSEを実装

教師なしSimCSEと教師ありSimCSEがある  
教師なしSimCSEの訓練には日本語のWIkipediaの記事本文から抽出した100万文のデータを使用  
教師ありSimCSEの訓練には日本語の自然言語推論データセットであるJSNLIを使用  

### 教師なしSimCSE

In [ ]:
!pip install datasets scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 23.5 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.

In [ ]:
from transformers.trainer_utils import set_seed
#乱数のシードを設定
set_seed(42)

In [ ]:
#データセットの読み込みと前処理
from datasets import load_dataset
unsup_train_dataset = load_dataset("llm-book/jawiki-sentences", split="train")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for llm-book/jawiki-sentences contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/jawiki-sentences
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [ ]:
print(unsup_train_dataset)

Dataset({
    features: ['text'],
    num_rows: 24387500
})


In [ ]:
for i, text in enumerate(unsup_train_dataset[:5]["text"]):
    print(i, text)

0 アンパサンド(&, 英語: ampersand)は、並立助詞「...と...」を意味する記号である。
1 ラテン語で「...と...」を表す接続詞 "et" の合字を起源とする。
2 現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" の合字であることが容易にわかる字形を使用している。
3 英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" のように唱えられていた。
4 また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。


In [ ]:
# 訓練セットから空行の事例を除外する
unsup_train_dataset = unsup_train_dataset.filter(lambda example: example["text"] != "")

Filter:   0%|          | 0/24387500 [00:00<?, ? examples/s]

In [ ]:
#訓練セットをシャッフルし、最初の100万事例を取り出す
unsup_train_dataset = unsup_train_dataset.shuffle().select(range(1000000))

In [ ]:
unsup_train_dataset = unsup_train_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [ ]:
# 文がシャッフルされていることを確認
for i, text in enumerate(unsup_train_dataset[:5]["text"]):
    print(i, text)

0 2005年の時点で、10,000人ものウズベキスタン人が韓国での労働に従事しており、その大部分が高麗人である。
1 小学5年生(11歳)の時から芸能活動を開始。
2 i ħ d d t | ψ ( t ) ⟩ = L ^ | ψ ( t ) ⟩ {\displaystyle i\hbar {\frac {d}{dt}}|\psi (t)\rangle ={\hat {L}}|\psi (t)\rangle }
3 安土宗論(あづちしゅうろん)は、1579年(天正7年)、安土城下の浄厳院で行われた浄土宗と法華宗の宗論。
4 1927年 オーストラリア選手権(1927ねんオーストラリアせんしゅけん、1927 Australian Championships)に関する記事。


In [ ]:
# JGLUEリポジトリからJSTSデータセットの訓練と検証を読み込み、それぞれをSimCSEの検証セットとテストセットとして使用する
valid_dataset = load_dataset("llm-book/JGLUE", name="JSTS", split="train")
test_dataset = load_dataset("llm-book/JGLUE", name="JSTS", split="validation")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for llm-book/JGLUE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/JGLUE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# トークナイザとcollate関数の準備
from transformers import AutoTokenizer
base_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [ ]:
# 訓練セットの事例に含まれる文のlistにトークナイザを適用した結果をSimCSEのモデルに入力する同一分のペアとして返す
import torch
from torch import Tensor
from transformers import BatchEncoding

# collate=照合
def unsup_train_collate_fn(examples: list[dict],) -> dict[str, BatchEncoding | Tensor]:
  """教師なしSimCSEの訓練セットのミニバッチを作成"""
  # ミニバッチに含まれる文にトークナイザを適用する
  tokenized_texts = tokenizer(
      [example["text"] for example in examples],
      padding=True,
      truncation=True,
      max_length=32,
      return_tensors="pt",
  )

  # 文と文の類似度行列における正例ペアの位置を示すTensorを作成する
  # i行目の文に対してi列目の文との組が正例ペアとなる
  labels = torch.arange(len(examples))

  return {
      "tokenized_texts_1": tokenized_texts,
      "tokenized_texts_2": tokenized_texts,
      "labels": labels,
  }

In [ ]:
# 検証、テストセットの事例に含まれる二つの文のそれぞれのlistにトークナイザを適用した結果をSimCSEのモデルに入力する文ペアとして返す
# データセットに付与された類似度スコアのラベルをlabel_scoresというTensorにして返す

def eval_collate_fn(examples: list[dict],) -> dict[str, BatchEncoding | Tensor]:
  """SimCSEの検証セットとテストセットのミニバッチを作成"""
  # ミニバッチの文ペアに含まれる文（文1と文2）のそれぞれにトークナイザを適用する
  tokenized_texts_1 = tokenizer(
      [example["sentence1"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt", #pytorchのテンソル型
  )

  tokenized_texts_2 = tokenizer(
      [example["sentence2"] for example in examples],
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="pt",
  )

# 文1と文2の類似度行列における正例ペアの位置を示すTensorを作成する
# 行列のi番目の事例（文1）に対してi列目の事例（文2）との組が正例ペアとなる
  labels = torch.arange(len(examples))

  # データセットに付与された類似度スコアのTensorを作成する
  label_scores = torch.tensor([example["label"] for example in examples])

  return {
      "tokenized_texts_1": tokenized_texts_1,
      "tokenized_texts_2": tokenized_texts_2,
      "labels": labels,
      "label_scores": label_scores,
  }



In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers.utils import ModelOutput

class SimCSEModel(nn.Module):
  """SimCSEのモデル"""

  def __init__(
      self,
      base_model_name: str,
      mlp_only_train:bool = False,
      temperature: float = 0.05,
  ) -> None:

    # モデルの初期化E
    super().__init__()

    # モデル名からエンコーダを初期化する
    self.encoder = AutoModel.from_pretrained(base_model_name)

    # MLP層の次元数
    self.hidden_size = self.encoder.config.hidden_size
    # MLP層の線形層
    self.dense = nn.Linear(self.hidden_size, self.hidden_size)
    # MLP層の活性化関数
    self.activation = nn.Tanh()

    # MLP層による変換を訓練時にのみ適用するよう設定するフラグ
    self.mlp_only_train = mlp_only_train
    # 交差エントロピー損失の計算時に使用する温度
    self.temperature = temperature

  def encode_texts(self, tokenized_texts:BatchEncoding) -> Tensor:
    """エンコーダを用いて文をベクトルに変換"""
    # トークナイズされた文をエンコーダに入力する
    encoded_texts = self.encoder(**tokenized_texts)
    # モデルの最終層の出力（last_hidden_state）の[CLS]トークン（0番目の位置のトークン）のベクトルを取り出す
    # エンコーダの出力を返す
    encoded_texts = encoded_texts.last_hidden_state[:, 0]

    # self.mlp_only_trainのフラグがTrueに設定されていて
    # かつ訓練時である場合、MLP層の変換を適用せずベクトルを返す
    if self.mlpp_only_train and self.training:
      return encoded_texts

    # MLP層によるベクトルの変換を行う
    encoded_texts = self.dense(encoded_texts)
    encoded_texts = self.activation(encoded_texts)

    return encoded_texts

  def forward(
      self,
      tokenized_texts_1: BatchEncoding,
      tokenized_texts_2: BatchEncoding,
      labels: Tensor,
      label_scores: Tensor | None = None,
  ) -> ModelOutput:
    """SimCSEのモデルのforward計算を定義"""
    # 文ペアをベクトルに変換する
    encoded_texts_1 = self.encode_texts(tokenized_texts_1)
    encoded_texts_2 = self.encode_texts(tokenized_texts_2)

    # 文ペアの類似度行列を計算する
    sim_matrix = F.cosine_similarity(
        encoded_texts_1.unsqueeze(1),
        encoded_texts_2.unsqueeze(0),
        dim=2,
        )

    # 交差エントロピー損失を計算する
    loss = F.cross_entropy(sim_matrix / self.temperature, labels)

    # 性能評価に使用するため、正例ペアに対するスコアを類似度行列から取り出す
    positive_mask = F.one_hot(labels, sim_matrix.size(1)).bool()
    positive_scores = torch.masked_select(
        sim_matrix, positive_mask
    )

    return ModelOutput(
        loss=loss,
        scores=positive_scores,
    )

# 教師なしSimCSEのモデルを初期化する
unsup_model = SimCSEModel(base_model_name, mlp_only_train=True)

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

In [ ]:
# モデルの評価結果を返す関数を定義
# 検証セットまたはテストセットのすべての事例（文ペア）に対して、モデルが出力するベクトルのコサイン類似度と、人手で付与された類似度スコアの間のスピアマンの順位相関係数を評価指標として値を出力
from scipy.stats import spearmanr
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction) -> dict[str, float]:
  """
  モデルが予測したスコアと評価用データのスコアのスピアマンの順位相関係数を計算
  """
  scores = p.predictions
  labels, label_scores = p.label_ids

  spearman = spearmanr(scores, label_scores).statistic

  return {"spearman": spearman}

In [ ]:
# 訓練時のハイパーパラメータをTrainingArgumentsクラスを用いて設定
# SimCSEの論文の設定に倣い、訓練を250ステップ行うごとに検証セットによる評価とロギングとチェックポイントの保存を行うよう
# evaluation_strategy = steps